# 阿里股价案例

In [1]:
import numpy as np
import pandas as pd

## 读取csv文档
csv文档头：date日期,close关闭价格,volume,open开盘价,high最高,low最低

用data作为索引

In [34]:
#最后一个字段是读取第一列、第二列意思
df = pd.read_csv('BABA_stock.csv', index_col='date', usecols=[0,1])  
df.head()

,close
date,
'2019/04/15',183.07
'2019/04/12',188.91
'2019/04/11',184.98
'2019/04/10',186.19
'2019/04/09',187.19


In [35]:
df.describe()

,close
count,755.000000
mean,143.366954
std,40.345464
min,74.230000
25%,102.925000
50%,152.110000
75%,179.155000
max,210.860000


## 把字符串日期转化为pandas的日期格式
注意：只是把index给改了，别的还是原始的

In [36]:
df.index = pd.DatetimeIndex(df.index.str.strip("'"))

In [37]:
df.index

DatetimeIndex(['2019-04-15', '2019-04-12', '2019-04-11', '2019-04-10',
               '2019-04-09', '2019-04-08', '2019-04-05', '2019-04-04',
               '2019-04-03', '2019-04-02',
               ...
               '2016-04-28', '2016-04-27', '2016-04-26', '2016-04-25',
               '2016-04-22', '2016-04-21', '2016-04-20', '2016-04-19',
               '2016-04-18', '2016-04-15'],
              dtype='datetime64[ns]', name='date', length=755, freq=None)

In [38]:
df.sort_index(inplace=True)

In [39]:
df.head()

,close
date,
2016-04-15,78.97
2016-04-18,79.01
2016-04-19,79.46
2016-04-20,81.21
2016-04-21,80.78


## day10 均线求法

In [50]:
#.dropna() 是把控制扔掉
# 取的是close价格那列
ma10 = df.rolling(10).mean().dropna()
ma10.head()

,close
date,
2016-04-28,79.082
2016-04-29,78.879
2016-05-02,78.639
2016-05-03,78.284
2016-05-04,77.745


## 判断大小

In [55]:
model = df['close'] - ma10['close'] > 0
model.head

<bound method NDFrame.head of date
2016-04-15    False
2016-04-18    False
2016-04-19    False
2016-04-20    False
2016-04-21    False
              ...  
2019-04-09     True
2019-04-10     True
2019-04-11     True
2019-04-12     True
2019-04-15    False
Name: close, Length: 755, dtype: bool>

## 自动以方法，找到拐点

In [56]:
# 定义函数方法1——仅判断买点
def get_deal_date(w):
    if w[0]==False and w[1]==True:
        return True
    else:
        return False

In [57]:
# 定义函数方法2——买点、卖点都融合，注意下边的写法——三目运算符
def get_deal_date(w, is_buy=True):
    if is_buy == True:
        return True if w[0] == False and w[1] == True else False
    else:
        return True if w[0] == True and w[1] == False else False

## 对model使用方法

In [73]:
# 这行代码是测试行——最后是转化成bool
model.rolling(2).apply(get_deal_date,raw=False).dropna().astype('bool')

date
2016-04-18    False
2016-04-19    False
2016-04-20    False
2016-04-21    False
2016-04-22    False
              ...  
2019-04-09    False
2019-04-10    False
2019-04-11    False
2019-04-12    False
2019-04-15    False
Name: close, Length: 754, dtype: bool

In [74]:
# 这行是真正使用的
se_buy = model.rolling(2).apply(get_deal_date, raw=False).fillna(0).astype('bool')
se_sell = model.rolling(2).apply(get_deal_date, raw=False, args=[False]).fillna(0).astype('bool')

In [70]:
se_sell.head(30)

date
2016-04-15    False
2016-04-18    False
2016-04-19    False
2016-04-20    False
2016-04-21    False
2016-04-22    False
2016-04-25    False
2016-04-26    False
2016-04-27    False
2016-04-28    False
2016-04-29    False
2016-05-02    False
2016-05-03    False
2016-05-04    False
2016-05-05    False
2016-05-06    False
2016-05-09    False
2016-05-10    False
2016-05-11    False
2016-05-12    False
2016-05-13     True
2016-05-16    False
2016-05-17    False
2016-05-18     True
2016-05-19    False
2016-05-20    False
2016-05-23    False
2016-05-24    False
2016-05-25     True
2016-05-26    False
Name: close, dtype: bool

## 索引到买卖价格

In [75]:
buy_info = df[se_buy.values]
sell_info = df[se_sell.values]
print(buy_info)
print(sell_info)

             close
date              
2016-05-05   78.83
2016-05-16   79.29
2016-05-23   79.00
2016-05-27   80.97
2016-06-14   77.77
...            ...
2019-01-24  155.86
2019-02-19  170.18
2019-03-18  181.83
2019-03-29  182.45
2019-04-04  181.07

[63 rows x 1 columns]
             close
date              
2016-05-13   77.16
2016-05-18   79.03
2016-05-25   75.59
2016-06-01   76.69
2016-06-17   77.00
...            ...
2019-02-14  168.38
2019-03-07  177.32
2019-03-22  176.26
2019-04-03  178.32
2019-04-15  183.07

[63 rows x 1 columns]


因为带日期没办法直接加减，因此改变一下索引方法

In [78]:
no_index_buy_info = buy_info.reset_index(drop=True)
no_index_sell_info = sell_info.reset_index(drop=True)
print(no_index_buy_info)
print(no_index_sell_info)

     close
0    78.83
1    79.29
2    79.00
3    80.97
4    77.77
..     ...
58  155.86
59  170.18
60  181.83
61  182.45
62  181.07

[63 rows x 1 columns]
     close
0    77.16
1    79.03
2    75.59
3    76.69
4    77.00
..     ...
58  168.38
59  177.32
60  176.26
61  178.32
62  183.07

[63 rows x 1 columns]


## 计算每次交易利润
按照每次买卖1股来计算的

In [79]:
profit = no_index_sell_info - no_index_buy_info
profit.describe()

,close
count,63.000000
mean,0.572540
std,6.419356
min,-9.190000
25%,-3.380000
50%,-1.120000
75%,3.435000
max,17.840000


In [80]:
profit.sum()  # 3年交易63次（上表），赚了36

close    36.07
dtype: float64

## 计算有1W用于交易能赚多少钱

In [85]:
all_money = 10000
remain = all_money
fee = 0.0005   #交易费百分比
for i in range(len(no_index_buy_info)):
    buy_count = remain / no_index_buy_info.iloc[i]
    remain = no_index_sell_info.iloc[i] * (1 - fee) * buy_count
    print(remain)

print(remain/all_money*100/3)

close    9783.257643
Name: 0, dtype: float64
close    9746.301754
dtype: float64
close    9320.944154
dtype: float64
close    8823.833463
dtype: float64
close    8732.100541
dtype: float64
close    8520.01106
dtype: float64
close    9067.899957
dtype: float64
close    10088.555854
dtype: float64
close    10655.960931
dtype: float64
close    10605.417999
dtype: float64
close    10504.975886
dtype: float64
close    10191.734094
dtype: float64
close    10072.952444
dtype: float64
close    9706.071546
dtype: float64
close    9583.513335
dtype: float64
close    10902.012235
dtype: float64
close    10688.246164
dtype: float64
close    10702.778181
dtype: float64
close    11143.066544
dtype: float64
close    12513.970447
dtype: float64
close    14206.752702
dtype: float64
close    13937.94529
dtype: float64
close    14858.208329
dtype: float64
close    14189.768087
dtype: float64
close    15438.483281
dtype: float64
close    15411.841643
dtype: float64
close    15008.545198
dtype: float64
clo

# 这个策略只是一个分析手段，并不能真金白银去投资